In [5]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [6]:
from pathlib import Path

base = Path("/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction")
print("Existe project root?", base.exists(), "-", base)
print("Existe data/raw?", (base / "data" / "raw").exists(), "-", (base / "data" / "raw"))


Existe project root? True - /content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction
Existe data/raw? True - /content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/raw


In [7]:
raw = Path("/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/raw")

if not raw.exists():
    print("❌ Pasta não encontrada:", raw)
else:
    files = sorted([p for p in raw.iterdir() if p.is_file()])
    print(f"Arquivos encontrados em {raw}:\n")
    for i, f in enumerate(files):
        print(f"[{i}] {f.name}")


Arquivos encontrados em /content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/raw:

[0] brasileirao_serie_a_2003_2024.csv
[1] football-data.co.uk.csv


In [8]:
from pathlib import Path
import pandas as pd
import numpy as np

RAW_DIR = Path("/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/raw")
FILE_PATH = RAW_DIR / "football-data.co.uk.csv"

# ---- Leitura robusta (encoding + separador) ----
def read_csv_robust(path: Path) -> pd.DataFrame:
    encodings = ["utf-8", "utf-8-sig", "latin1", "cp1252"]
    seps = [",", ";", "\t", "|"]
    last_err = None

    for enc in encodings:
        for sep in seps:
            try:
                df = pd.read_csv(path, encoding=enc, sep=sep)
                if df.shape[1] > 1:
                    print(f"✅ Lido com encoding='{enc}' e sep='{sep}'")
                    return df
            except Exception as e:
                last_err = e
    raise RuntimeError(f"Falha ao ler o CSV. Último erro: {last_err}")

df_fd = read_csv_robust(FILE_PATH)

# ---- Visão geral ----
print("\n--- VISÃO GERAL ---")
print("Arquivo:", FILE_PATH.name)
print("Shape:", df_fd.shape)
print("\nColunas:")
print(df_fd.columns.tolist())

print("\n--- HEAD (5) ---")
display(df_fd.head())

print("\n--- INFO ---")
df_fd.info()

print("\n--- % NULOS (top 25) ---")
display(df_fd.isnull().mean().sort_values(ascending=False).head(25))

# ---- Detectar colunas prováveis (heurística) ----
def find_col(possibles):
    # match exato
    for p in possibles:
        for c in df_fd.columns:
            if c.lower() == p.lower():
                return c
    # match parcial
    for p in possibles:
        for c in df_fd.columns:
            if p.lower() in c.lower():
                return c
    return None

col_date = find_col(["date", "data", "matchdate"])
col_home = find_col(["hometeam", "home_team", "home"])
col_away = find_col(["awayteam", "away_team", "away"])
col_hg   = find_col(["fthg", "hg", "homegoals", "gols_mandante"])
col_ag   = find_col(["ftag", "ag", "awaygoals", "gols_visitante"])
col_res  = find_col(["ftr", "result", "resultado"])

print("\n--- COLUNAS DETECTADAS (heurística) ---")
print("Data:", col_date)
print("Mandante:", col_home)
print("Visitante:", col_away)
print("Gols mandante:", col_hg)
print("Gols visitante:", col_ag)
print("Resultado:", col_res)

# ---- Parse de data + range ----
if col_date:
    dt = pd.to_datetime(df_fd[col_date], errors="coerce", dayfirst=True)
    print("\n--- PARSE DE DATA ---")
    print("Datas inválidas:", int(dt.isna().sum()))
    if dt.notna().any():
        print("Min:", dt.min(), "| Max:", dt.max())
else:
    print("\n⚠️ Não encontrei coluna de data automaticamente. Mostre o df_fd.columns que eu ajusto.")

# ---- Se houver placares, checar consistência básica ----
if col_hg and col_ag:
    # converter para numérico (se vier como string)
    df_fd[col_hg] = pd.to_numeric(df_fd[col_hg], errors="coerce")
    df_fd[col_ag] = pd.to_numeric(df_fd[col_ag], errors="coerce")
    print("\n--- CHECK PLACARES ---")
    print("Nulos em gols mandante:", int(df_fd[col_hg].isna().sum()))
    print("Nulos em gols visitante:", int(df_fd[col_ag].isna().sum()))
    print("Faixa gols mandante:", (df_fd[col_hg].min(), df_fd[col_hg].max()))
    print("Faixa gols visitante:", (df_fd[col_ag].min(), df_fd[col_ag].max()))

# ---- Checar duplicatas lógicas (se tiver data+times) ----
if col_date and col_home and col_away:
    dups = df_fd.duplicated(subset=[col_date, col_home, col_away]).sum()
    print("\n--- DUPLICATAS LÓGICAS ---")
    print("Duplicatas por (data, mandante, visitante):", int(dups))

# ---- Checar cobertura de 2024 (se tiver data) ----
if col_date:
    df_fd["_date"] = pd.to_datetime(df_fd[col_date], errors="coerce", dayfirst=True)
    df_2024 = df_fd[df_fd["_date"].dt.year == 2024].copy()
    print("\n--- COBERTURA 2024 ---")
    print("Linhas em 2024:", df_2024.shape[0])
    if df_2024.shape[0] > 0 and col_home and col_away:
        print("Times mandante (top 10):")
        display(df_2024[col_home].value_counts().head(10))
        print("Times visitante (top 10):")
        display(df_2024[col_away].value_counts().head(10))


✅ Lido com encoding='utf-8' e sep=','

--- VISÃO GERAL ---
Arquivo: football-data.co.uk.csv
Shape: (5320, 25)

Colunas:
['Country', 'League', 'Season', 'Date', 'Time', 'Home', 'Away', 'HG', 'AG', 'Res', 'PSCH', 'PSCD', 'PSCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'BFECH', 'BFECD', 'BFECA', 'B365CH', 'B365CD', 'B36CA']

--- HEAD (5) ---


,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,...,MaxCA,AvgCH,AvgCD,AvgCA,BFECH,BFECD,BFECA,B365CH,B365CD,B36CA
0,Brazil,Serie A,2012,19/05/2012,22:30,Palmeiras,Portuguesa,1.0,1.0,D,...,5.31,1.69,3.50,4.90,NaN,NaN,NaN,NaN,NaN,NaN
1,Brazil,Serie A,2012,19/05/2012,22:30,Sport Recife,Flamengo RJ,1.0,1.0,D,...,2.70,2.59,3.23,2.58,NaN,NaN,NaN,NaN,NaN,NaN
2,Brazil,Serie A,2012,20/05/2012,01:00,Figueirense,Nautico,2.0,1.0,H,...,7.22,1.59,3.67,5.64,NaN,NaN,NaN,NaN,NaN,NaN
3,Brazil,Serie A,2012,20/05/2012,20:00,Botafogo RJ,Sao Paulo,4.0,2.0,H,...,3.15,2.35,3.26,2.84,NaN,NaN,NaN,NaN,NaN,NaN
4,Brazil,Serie A,2012,20/05/2012,20:00,Corinthians,Fluminense,0.0,1.0,A,...,4.41,1.89,3.33,3.89,NaN,NaN,NaN,NaN,NaN,NaN



--- INFO ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5320 entries, 0 to 5319
Data columns (total 25 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  5320 non-null   object 
 1   League   5320 non-null   object 
 2   Season   5320 non-null   int64  
 3   Date     5320 non-null   object 
 4   Time     5320 non-null   object 
 5   Home     5320 non-null   object 
 6   Away     5320 non-null   object 
 7   HG       5319 non-null   float64
 8   AG       5319 non-null   float64
 9   Res      5319 non-null   object 
 10  PSCH     5275 non-null   float64
 11  PSCD     5275 non-null   float64
 12  PSCA     5275 non-null   float64
 13  MaxCH    5320 non-null   float64
 14  MaxCD    5320 non-null   float64
 15  MaxCA    5320 non-null   float64
 16  AvgCH    5320 non-null   float64
 17  AvgCD    5320 non-null   float64
 18  AvgCA    5320 non-null   float64
 19  BFECH    615 non-null    float64
 20  BFECD    615 non-null    float64
 21  

,0
B36CA,0.962030
B365CH,0.962030
B365CD,0.962030
BFECD,0.884398
BFECA,0.884398
BFECH,0.884398
PSCH,0.008459
PSCD,0.008459
PSCA,0.008459
HG,0.000188



--- COLUNAS DETECTADAS (heurística) ---
Data: Date
Mandante: Home
Visitante: Away
Gols mandante: HG
Gols visitante: AG
Resultado: None

--- PARSE DE DATA ---
Datas inválidas: 0
Min: 2012-05-19 00:00:00 | Max: 2025-12-07 00:00:00

--- CHECK PLACARES ---
Nulos em gols mandante: 1
Nulos em gols visitante: 1
Faixa gols mandante: (0.0, 8.0)
Faixa gols visitante: (0.0, 6.0)

--- DUPLICATAS LÓGICAS ---
Duplicatas por (data, mandante, visitante): 0

--- COBERTURA 2024 ---
Linhas em 2024: 380
Times mandante (top 10):


,count
Home,
Criciuma,19
Internacional,19
Fluminense,19
Sao Paulo,19
Athletico-PR,19
Atletico GO,19
Corinthians,19
Vasco,19
Cruzeiro,19


Times visitante (top 10):


,count
Away,
Juventude,19
Bahia,19
Bragantino,19
Fortaleza,19
Cuiaba,19
Flamengo RJ,19
Atletico-MG,19
Gremio,19
Botafogo RJ,19


In [9]:
df_fd["Season"].value_counts().sort_index()


,count
Season,
2012,380
2013,380
2014,380
2015,380
2016,380
2017,380
2018,380
2019,380
2020,380


In [10]:
# Jogos de 2025 com placar REAL
df_2025 = df_fd[df_fd["Season"] == 2025].copy()

df_2025["Date"] = pd.to_datetime(df_2025["Date"], dayfirst=True)

# Quantos jogos têm placar preenchido?
print("HG nulos:", df_2025["HG"].isna().sum())
print("AG nulos:", df_2025["AG"].isna().sum())
print("Res nulos:", df_2025["Res"].isna().sum())

# Datas mín / máx
print("Min date:", df_2025["Date"].min())
print("Max date:", df_2025["Date"].max())

# Amostra de jogos SEM placar
display(df_2025[df_2025["HG"].isna()].head(5))


HG nulos: 0
AG nulos: 0
Res nulos: 0
Min date: 2025-03-29 00:00:00
Max date: 2025-12-07 00:00:00


,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,...,AvgCH,AvgCD,AvgCA,BFECH,BFECD,BFECA,B365CH,B365CD,B36CA,_date


In [11]:
import pandas as pd

# Garantir datetime
df_fd["Date"] = pd.to_datetime(df_fd["Date"], dayfirst=True)

# Filtrar temporada 2025
df_2025 = df_fd[df_fd["Season"] == 2025].copy()

# Estrutura base por jogo
home = df_2025[["Home", "Away", "HG", "AG", "Res"]].copy()
away = df_2025[["Home", "Away", "HG", "AG", "Res"]].copy()

# Mandante
home["team"] = home["Home"]
home["points"] = home["Res"].map({"H": 3, "D": 1, "A": 0})
home["wins"] = (home["Res"] == "H").astype(int)
home["draws"] = (home["Res"] == "D").astype(int)
home["losses"] = (home["Res"] == "A").astype(int)
home["gf"] = home["HG"]
home["ga"] = home["AG"]

# Visitante
away["team"] = away["Away"]
away["points"] = away["Res"].map({"A": 3, "D": 1, "H": 0})
away["wins"] = (away["Res"] == "A").astype(int)
away["draws"] = (away["Res"] == "D").astype(int)
away["losses"] = (away["Res"] == "H").astype(int)
away["gf"] = away["AG"]
away["ga"] = away["HG"]

# Consolidar
cols = ["team", "points", "wins", "draws", "losses", "gf", "ga"]
table = pd.concat([home[cols], away[cols]])

# Agregar por time
league_table = (
    table
    .groupby("team", as_index=False)
    .sum()
)

# Métricas finais
league_table["gd"] = league_table["gf"] - league_table["ga"]
league_table["played"] = (
    league_table["wins"] + league_table["draws"] + league_table["losses"]
)

# Ordenação padrão Brasileirão
league_table = league_table.sort_values(
    by=["points", "wins", "gd", "gf"],
    ascending=[False, False, False, False]
).reset_index(drop=True)

# Adicionar posição
league_table.insert(0, "position", league_table.index + 1)

# Exibir
display(league_table)


,position,team,points,wins,draws,losses,gf,ga,gd,played
0,1,Flamengo RJ,79,23,10,5,78.0,27.0,51.0,38
1,2,Palmeiras,76,23,7,8,66.0,33.0,33.0,38
2,3,Cruzeiro,70,19,13,6,55.0,31.0,24.0,38
3,4,Mirassol,67,18,13,7,63.0,39.0,24.0,38
4,5,Fluminense,64,19,7,12,50.0,39.0,11.0,38
5,6,Botafogo RJ,63,17,12,9,58.0,38.0,20.0,38
6,7,Bahia,60,17,9,12,50.0,46.0,4.0,38
7,8,Sao Paulo,51,14,9,15,43.0,47.0,-4.0,38
8,9,Gremio,49,13,10,15,47.0,50.0,-3.0,38
9,10,Bragantino,48,14,6,18,45.0,57.0,-12.0,38


In [12]:
import pandas as pd
from pathlib import Path

RAW_DIR = Path("/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/raw")
INTERIM_DIR = Path("/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/interim")
INTERIM_DIR.mkdir(parents=True, exist_ok=True)

df_fd = pd.read_csv(RAW_DIR / "football-data.co.uk.csv")

df_fd = df_fd.rename(columns={
    "Season": "ano_campeonato",
    "Date": "data",
    "Home": "time_mandante",
    "Away": "time_visitante",
    "HG": "gols_mandante",
    "AG": "gols_visitante",
    "Res": "resultado"
})

df_fd["data"] = pd.to_datetime(df_fd["data"], dayfirst=True, errors="coerce")
df_fd["gols_mandante"] = pd.to_numeric(df_fd["gols_mandante"], errors="coerce")
df_fd["gols_visitante"] = pd.to_numeric(df_fd["gols_visitante"], errors="coerce")

fd_base = df_fd[[
    "ano_campeonato", "data", "time_mandante", "time_visitante",
    "gols_mandante", "gols_visitante", "resultado"
]].dropna().copy()

fd_train = fd_base[fd_base["ano_campeonato"].between(2012, 2023)].copy()
fd_test  = fd_base[fd_base["ano_campeonato"] == 2024].copy()
fd_2025  = fd_base[fd_base["ano_campeonato"] == 2025].copy()

print("FD train:", fd_train.shape)
print("FD test:", fd_test.shape)
print("FD 2025:", fd_2025.shape)

fd_train.to_parquet(INTERIM_DIR / "fd_2012_2023.parquet", index=False)
fd_test.to_parquet(INTERIM_DIR / "fd_2024.parquet", index=False)
fd_2025.to_parquet(INTERIM_DIR / "fd_2025.parquet", index=False)

print("✅ football-data parquets salvos em:", INTERIM_DIR)


FD train: (4559, 7)
FD test: (380, 7)
FD 2025: (380, 7)
✅ football-data parquets salvos em: /content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/interim
